<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/sglogo.png width=50  align="left">

# Encryption as a Service using MySQL
Based on
https://github.com/assareh/transit-app-example

**Requirements:  Enterprise Vault**  
Or simply run `evrd1`   
---

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
# apply Enterprise License

export VAULT_ROOT_TOKEN=root
curl --header "X-Vault-Token: $VAULT_ROOT_TOKEN" \
    --insecure \
    --request PUT \
    --data @/Users/tio/Documents/vault_license.json \
    $VAULT_ADDR/v1/sys/license

In [ ]:
unset VAULT_TOKEN; vault login root

In [ ]:
vault status; vault version

**Create a database**  

In [ ]:
## one time thing
docker pull mysql/mysql-server:5.7.21

In [5]:
mkdir /tmp/workshop-data

In [7]:
#  -v /tmp/workshop-data:/var/lib/mysql \
docker run --rm --name workshop-mysql \
  -p 3306:3306 \
  -e MYSQL_ROOT_PASSWORD=root \
  -e MYSQL_ROOT_HOST=% \
  -e MYSQL_DATABASE=my_app \
  -e MYSQL_USER=vault \
  -e MYSQL_PASSWORD=vaultpw \
  -d mysql/mysql-server:5.7.21

0a4e9653ddae49a3cc43799b5284cb0ee10f481474971fe30b6878336a3aebdb


**Start the database**

In [8]:
docker ps

CONTAINER ID   IMAGE                       COMMAND                  CREATED          STATUS                             PORTS                               NAMES
0a4e9653ddae   mysql/mysql-server:5.7.21   "/entrypoint.sh mysq…"   25 seconds ago   Up 24 seconds (health: starting)   0.0.0.0:3306->3306/tcp, 33060/tcp   workshop-mysql
9b8d32ee26c5   nginx:1.13.6-alpine         "nginx -g 'daemon of…"   2 weeks ago      Up 21 minutes                      80/tcp, 0.0.0.0:8443->443/tcp       nginx_proxy
1124a54fb16e   cyberark/conjur             "conjurctl server"       2 weeks ago      Up 6 seconds                       80/tcp                              conjur_server
3f4eef754b3c   kindest/node:v1.20.2        "/usr/local/bin/entr…"   3 weeks ago      Up 21 minutes                      127.0.0.1:51133->6443/tcp           terraform-learn-control-plane


In [4]:
docker exec -it workshop-mysql mysql -uroot -proot -e 'show grants for root'

Error response from daemon: Container 0e605a3a7d10779ea48fe2b4eb29a6848cb12042326c12eb29b2498a78a8684e is not running


**Display existing database users**

In [ ]:
docker exec -it workshop-mysql mysql -uroot -proot -e  "select user from mysql.user;"

In [ ]:
docker exec -it workshop-mysql mysql -uroot -proot -e "show databases"

### Configure Vault and Database Engine

In [ ]:
vault audit enable file file_path=/tmp/audit/vault_audit.log

In [ ]:
vault secrets enable -path=lob_a/workshop/database database

**Configure Vault `workshop-app` role**

In [ ]:
vault write lob_a/workshop/database/config/ws-mysql-database \
    plugin_name=mysql-database-plugin \
    connection_url="{{username}}:{{password}}@tcp(host.docker.internal:3306)/" \
    allowed_roles="workshop-app-long,workshop-app" \
    username="root" \
    password="root"

In [ ]:
vault write lob_a/workshop/database/roles/workshop-app-long \
    db_name=ws-mysql-database \
    creation_statements="CREATE USER '{{name}}'@'%' IDENTIFIED BY '{{password}}';GRANT ALL ON *.* TO '{{name}}'@'%';" \
    default_ttl="1h" \
    max_ttl="24h"

In [ ]:
vault write lob_a/workshop/database/roles/workshop-app \
    db_name=ws-mysql-database \
    creation_statements="CREATE USER '{{name}}'@'%' IDENTIFIED BY '{{password}}';GRANT ALL ON *.* TO '{{name}}'@'%';" \
    default_ttl="5m" \
    max_ttl="1h"

### Demo
---

### Get Database credential

In [ ]:
vault read lob_a/workshop/database/creds/workshop-app-long

&nbsp;

   
**Display database users** (note new user added by Vault)

In [ ]:
docker exec -it workshop-mysql mysql -u root  -p'root' \
  -e "select user from mysql.user"; 

**Database test using the newly created credential**

In [ ]:
docker exec -it workshop-mysql mysql -u v_token_workshop-a_I0zwAQcm8Tiiu  -p'SYDne2acJSzmmJs-YuG4' \
  -e "show databases";

### Configure Encryption as a Service 

In [ ]:
vault secrets enable -path=lob_a/workshop/transit transit

In [ ]:
vault write -f lob_a/workshop/transit/keys/customer-key

In [ ]:
vault write -f lob_a/workshop/transit/keys/archive-key

### Configure Transform

In [ ]:
vault secrets enable -path=lob_a/workshop/transform transform

In [ ]:
vault write lob_a/workshop/transform/role/ssn transformations=ssn-fpe

In [ ]:
vault write lob_a/workshop/transform/transformation/ssn-fpe \
  type=fpe \
  template=builtin/socialsecuritynumber \
  tweak_source=internal \
  allowed_roles=ssn

# RUN the transit app 
**Note:** after a while the transit-app-example gives 500 Internal Server Error.  Just stop and rerun the container.

In [12]:
export LAPTOP_IP=$(ipconfig getifaddr en0)
echo "Transit-App is going to run on http://$LAPTOP_IP:5000\n"

docker run --rm --name transit-app-example \
  -p 5000:5000 \
  -e VAULT_ADDR=http://host.docker.internal:8200 \
  -e VAULT_DATABASE_CREDS_PATH=lob_a/workshop/database/creds/workshop-app-long \
  -e VAULT_NAMESPACE=root \
  -e VAULT_TOKEN=$(vault print token) \
  -e VAULT_TRANSFORM_PATH=lob_a/workshop/transform \
  -e VAULT_TRANSFORM_MASKING_PATH=lob_a/workshop/transform \
  -e VAULT_TRANSIT_PATH=lob_a/workshop/transit \
  -e MYSQL_ADDR=host.docker.internal \
  -d assareh/transit-app-example:latest

Transit-App is going to run on http://192.168.1.73:5000

afcbf38ccd5cff20a096ea13461ac996bbb7a70ca0f75a66a8a7a34a4385ac0b


---
### Cleanup


**Shutdown Transit-app**

In [13]:
docker stop transit-app-example

transit-app-example


**Shutdown mysql**

In [ ]:
docker stop workshop-mysql

In [ ]:
docker rm workshop-mysql

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">

### Using Transit-App on your laptop
Follow Setup instructions on https://github.com/assareh/transit-app-example/tree/docker/setup  

**NOTE**: instead of running  
`pip3 install mysql-connector-python hvac Flask` 
use   
`python3 -m pip install mysql-connector-python hvac Flask` .  This uses the pip that comes with your particular python3 version.

In [ ]:
export LAPTOP_IP=$(ipconfig getifaddr en0)
echo "Transit-App is going to run on http://$LAPTOP_IP:5000\n"

export VAULT_TOKEN=root
export VAULT_ADDR=http://$LAPTOP_IP:8200
export VAULT_NAMESPACE=root
export VAULT_TRANSIT_PATH=lob_a/workshop/transit
export VAULT_TRANSFORM_PATH=lob_a/workshop/transform
export VAULT_TRANSFORM_MASKING_PATH=lob_a/workshop/transform
export VAULT_DATABASE_CREDS_PATH=lob_a/workshop/database/creds/workshop-app-long
export MYSQL_ADDR=$LAPTOP_IP

cd ~/mydemo/vault/transit-app-example/backend/
python3 app.py